#  3.6. Implementation of Softmax Regression from Scratch

Así como implementamos la regresión lineal desde cero, creemos que la regresión softmax es igualmente fundamental y debe conocer los detalles sangrientos de

cómo implementarlo usted mismo. Trabajaremos con el conjunto de datos Fashion-MNIST, recién presentado en la Sección 3.5 , configurando un iterador de datos con tamaño de lote 256.


In [1]:

# Importacion de librerias


use strict;
use warnings;
use Data::Dump qw(dump);
use List::Util qw( min max shuffle);
# use constant np => 'numperl';
# use constant ex => 'extension';
# use constant False => 'False';
# use constant True => 'True';
# use constant None => 'None';
use AI::MXNet qw(mx);
# use AI::MXNet qw(nd);
use Data::Dump qw(dump);
use AI::MXNet::AutoGrad qw(autograd);
use Chart::Plotly::Trace::Scatter;

use d2l qw(d2l);
use d2l::Accumulator qw(accumulator);
use d2l::Animator qw(animator);
IPerl->load_plugin('Chart::Plotly');



In [2]:
# no funciona completamente

#my $d2l = d2l->new();

my $batch_size = 256;
my ($train_iter, $test_iter) = d2l->load_data_fashion_mnist($batch_size);

Error: Operation """": no method found, argument in overloaded package AI::MXNet::Gluon::Data::Loader::DataLoader at /usr/local/lib/perl5/site_perl/5.32.1/Reply/Plugin/Defaults.pm line 86.


# 3.6.1. Inicialización de los parámetros del modelo

Como en nuestro ejemplo de regresión lineal, cada ejemplo aquí estará representado por un vector de longitud fija. Cada ejemplo en el conjunto de datos sin procesar es un 28×28 imagen. En esta sección, aplanaremos cada imagen, tratándolas como vectores de longitud 784. En el futuro, hablaremos sobre estrategias más sofisticadas para explotar la estructura espacial en las imágenes, pero por ahora trataremos cada ubicación de píxel como una característica más.

Recuerde que en la regresión softmax, tenemos tantas salidas como clases. Debido a que nuestro conjunto de datos tiene 10 clases, nuestra red tendrá una dimensión de salida de 10. En consecuencia, nuestros pesos constituirán un  784×10 matriz y los sesgos constituirán una  1×10
 vector fila. Al igual que con la regresión lineal, inicializaremos nuestros pesos Wcon ruido gaussiano y nuestros sesgos para tomar el valor inicial 0.

In [3]:
# funciona completamente

my $n_inputs = 784;
my $n_outputs = 10;

our $w = mx->nd->random_normal(0, 0.01, [$n_inputs, $n_outputs]);
our $b = mx->nd->zeros([$n_outputs]);
$w->attach_grad();
$b->attach_grad();

# 3.6.2. Definición de la operación Softmax

Antes de implementar el modelo de regresión softmax, revisemos brevemente cómo funciona el operador de suma a lo largo de dimensiones específicas en un tensor, como se analiza en la Sección 2.3.6 y la Sección 2.3.6.1 . Dada una matriz X, podemos sumar sobre todos los elementos (por defecto) o solo sobre los elementos en el mismo eje, es decir, la misma columna (eje 0) o la misma fila (eje 1). Tenga en cuenta que si Xes un tensor con forma (2, 3) y sumamos sobre las columnas, el resultado será un vector con forma (3,). Al invocar el operador de suma, podemos especificar que se mantenga el número de ejes en el tensor original, en lugar de colapsar la dimensión que sumamos. Esto dará como resultado un tensor bidimensional con forma (1, 3).

In [4]:
# funciona completamente

my $x = mx->nd->array([[1,2,3],[4,5,6]]);

print(dump($x->sum(0)->asarray));
print(dump($x->sum(1)->asarray));

[5, 7, 9][6, 15]

1

Ahora estamos listos para implementar la operación softmax. Recuerde que softmax consta de tres pasos: i) exponenciamos cada término (usando exp); ii) sumamos sobre cada fila (tenemos una fila por ejemplo en el lote) para obtener la constante de normalización para cada ejemplo; iii) dividimos cada fila por su constante de normalización, asegurándonos de que el resultado sume 1. Antes de mirar el código, recordemos cómo se ve esto expresado como una ecuación:

(3.6.1)

$$ softmax(X)ij=exp(Xij)∑kexp(Xik). $$
 
El denominador, o constante de normalización, también se denomina a veces función de partición (y su logaritmo se denomina función de partición logarítmica). Los orígenes de ese nombre se encuentran en la física estadística, donde una ecuación relacionada modela la distribución sobre un conjunto de partículas.

In [5]:
# funciona completamente

sub softmax {
    my $x = shift;
    my $x_exp = mx->nd->exp($x);
    my $partition = $x_exp->sum(1);
    my $return_value = $x_exp->transpose / $partition->transpose;
    return $return_value->transpose;
}

Como puede ver, para cualquier entrada aleatoria, convertimos cada elemento en un número no negativo. Además, cada fila suma 1, como se requiere para una probabilidad.

In [6]:
# funciona completamente

my $sm_test = mx->nd->random_normal(0, 1, [2,5]);
my $x_prob = softmax($sm_test);
print(dump($x_prob->asarray));
print(dump($x_prob->sum(1)->asarray));

[
  [
    0.213241934776306,
    0.339617758989334,
    0.123974204063416,
    0.271060973405838,
    0.0521052107214928,
  ],
  [
    0.114622637629509,
    0.346123397350311,
    0.194010332226753,
    0.295833259820938,
    0.0494103617966175,
  ],
][1.00000011920929, 1]

1

Tenga en cuenta que si bien esto parece correcto matemáticamente, fuimos un poco descuidados en nuestra implementación porque no tomamos precauciones contra el desbordamiento o subdesbordamiento numérico debido a elementos grandes o muy pequeños de la matriz.

# 3.6.3. Definición del modelo

Ahora que hemos definido la operación softmax, podemos implementar el modelo de regresión softmax. El siguiente código define cómo se asigna la entrada a la salida a través de la red. Tenga en cuenta que aplanamos cada imagen original en el lote en un vector usando la reshapefunción antes de pasar los datos a través de nuestro modelo.

In [8]:
# funciona completamente
sub net {
    my ($x, $w, $b) = (shift, shift, shift);
    $x = $x->reshape([-1, ($w->shape)->[0]]);
    my $product = mx->nd->dot($x, $w);
    return $product + $b;
}
#our $net = net($x, $w, $b);

Warning: Subroutine net redefined at reply input line 2.


# 3.6.4. Definición de la función de pérdida

A continuación, necesitamos implementar la función de pérdida de entropía cruzada, como se presentó en la Sección 3.4 . Esta puede ser la función de pérdida más común en todo el aprendizaje profundo porque, en este momento, los problemas de clasificación superan en número a los problemas de regresión.

Recuerde que la entropía cruzada toma la verosimilitud logarítmica negativa de la probabilidad predicha asignada a la etiqueta verdadera. En lugar de iterar sobre las predicciones con un ciclo for de Python (que tiende a ser ineficiente), podemos seleccionar todos los elementos con un solo operador. A continuación, creamos datos de muestra y_hatcon 2 ejemplos de probabilidades predichas sobre 3 clases y sus etiquetas correspondientes y. Sabemos yque en el primer ejemplo la primera clase es la predicción correcta y en el segundo ejemplo la tercera clase es la verdad fundamental. Usando y como índices de las probabilidades en y_hat, elegimos la probabilidad de la primera clase en el primer ejemplo y la probabilidad de la tercera clase en el segundo ejemplo.

In [9]:
# funciona completamente

my $y = mx->nd->array([0,2]);
my $y_hat = mx->nd->array([[0.1,0.3,0.6],[0.3,0.2,0.5]]);
my $aux = mx->nd->array([0, 1]);
print dump $y_hat->slice([$aux, $y])->asarray;


[0.100000001490116, 0.5]

1

Ahora podemos implementar la función de pérdida de entropía cruzada de manera eficiente con solo una línea de código.

In [10]:

# funciona completamente

sub cross_entropy{
    my ($y_hat ,$y) = @_;
    my $y_hat_range = mx->nd->arange(stop => $y_hat->len);
    my $result = (mx->nd->log($y_hat->slice([$y_hat_range, $y]))) * -1;
    
    return $result;
}

my $cross_entropy = cross_entropy($y_hat, $y);

print dump $cross_entropy->asarray;

[2.30258512496948, 0.6931471824646]

1

# 3.6.5. Precisión de clasificación

Dada la distribución de probabilidad pronosticada y_hat, normalmente elegimos la clase con la probabilidad pronosticada más alta cada vez que debemos generar una predicción sólida. De hecho, muchas aplicaciones requieren que hagamos una elección. Gmail debe clasificar un correo electrónico en "Principal", "Social", "Actualizaciones" o "Foros". Puede estimar probabilidades internamente, pero al final del día tiene que elegir una entre las clases.

Cuando las predicciones son consistentes con la clase de etiqueta y, son correctas. La precisión de clasificación es la fracción de todas las predicciones que son correctas. Aunque puede ser difícil optimizar la precisión directamente (no es diferenciable), a menudo es la medida de rendimiento que más nos importa y casi siempre la informaremos cuando entrenemos clasificadores.

Para calcular la precisión hacemos lo siguiente. Primero, si y_hates una matriz, asumimos que la segunda dimensión almacena puntajes de predicción para cada clase. Usamos argmaxpara obtener la clase pronosticada por el índice de la entrada más grande en cada fila. Luego comparamos la clase pronosticada con la verdad yfundamental por elementos. Dado que el operador de igualdad ==es sensible a los tipos de datos, convertimos y_hatel tipo de datos de para que coincida con el de y. El resultado es un tensor que contiene entradas de 0 (falso) y 1 (verdadero). Tomando la suma se obtiene el número de predicciones correctas.

In [11]:
# funciona completamente
sub accuracy{
      my($y_hat, $y) = @_;
     
     if (scalar @{$y_hat->shape} > 1 && $y_hat->shape->[1] > 1){
        $y_hat = $y_hat->argmax(axis => 1);
     
     }
     my $cmp = ($y_hat->astype($y->dtype) == $y);
     return $cmp->sum->asscalar;
    }


Continuaremos usando las variables y_haty ydefinidas antes como las distribuciones de probabilidad pronosticadas y las etiquetas, respectivamente. Podemos ver que la clase de predicción del primer ejemplo es 2 (el elemento más grande de la fila es 0.6 con el índice 2), lo cual es inconsistente con la etiqueta real, 0. La clase de predicción del segundo ejemplo es 2 (el elemento más grande de la fila es 0,5 con el índice de 2), que es consistente con la etiqueta real, 2. Por lo tanto, la tasa de precisión de clasificación para estos dos ejemplos es 0,5.

In [12]:
# Este tiene error 

print dump accuracy($y_hat, $y) / $y->len;

0.5

1

De manera similar, podemos evaluar la precisión de cualquier modelo neten un conjunto de datos al que se accede a través del iterador de datos data_iter.

In [13]:
sub evaluate_accuracy{

    my($net, $data_iter ) = @_;
    my $metric = Accumulator->new(2);
    
    my ($X , $y ,$accuracy);
    while(defined(my $batch = <$data_iter>)){
    
    $X = $batch->[0];
    $y = $batch->[1]->astype('float32');
    $accuracy = accuracy($net->($X), $y);
    
    $metric->add([ $accuracy, $y->size ]);
    print "No. of correct predictions:" ,  $metric->getitem(0) , "No of predict" , $metric->getitem(1);
    return ($metric->getitem(0) / $metric->getitem(1));    
}
}

Aquí Accumulatorhay una clase de utilidad para acumular sumas sobre múltiples variables. En la evaluate_accuracyfunción anterior, creamos 2 variables en la Accumulatorinstancia para almacenar tanto el número de predicciones correctas como el número de predicciones, respectivamente. Ambos se acumularán con el tiempo a medida que iteramos sobre el conjunto de datos.

In [ ]:
# package Accumulator{

    #sub __init__{ 
      
    #}
    #sub add{
     
    
    #}
    #sub _reset_{
    
    
   # }
  #  sub __getitem__{
    
    
 #   }
#}

Debido a que inicializamos el netmodelo con pesos aleatorios, la precisión de este modelo debería estar cerca de la suposición aleatoria, es decir, 0,1 para 10 clases.

In [14]:
print evaluate_accuracy(\&net, $test_iter);

Error: Can't call method "shape" on an undefined value at reply input line 4.


# 3.6.6. Capacitación
El ciclo de entrenamiento para la regresión softmax debería parecer sorprendentemente familiar si lee nuestra implementación de la regresión lineal en la Sección 3.2 . Aquí refactorizamos la implementación para que sea reutilizable. Primero, definimos una función para entrenar para una época. Tenga en cuenta que updateres una función general para actualizar los parámetros del modelo, que acepta el tamaño del lote como argumento. Puede ser un contenedor de la d2l.sgdfunción o una función de optimización integrada del marco.

In [15]:
sub train_epoch_ch3{

    my ($net, $train_iter, $loss, $updater, $batch_size) = @_;
    my $metric= Accumulator->new(3);
    
    if(ref($updater) eq 'AI::MXNet::Gluon::Trainer'){
    
        $updater->step($batch_size, 1);
        
    }
    my ($X ,$y, $y_hat, $l);
    while(defined(my $batch = <$train_iter> )){
    
        $X = $batch->[0];
        $y = $batch->[1]->astype('float32');
        
        autograd->record(sub{
        
            $y_hat = $net->($X);
            $l = $loss->($y_hat, $y);
        });
        
        $l->backward();
        if(ref($updater) eq 'AI::MXNet::Gluon::Trainer'){
        
            $updater->step($batch_size);
            
        }
        else{
        
            $updater->($batch_size);
        }
        $metric->add([ $l->sum->asscalar, accuracy($y_hat, $y), $y->size]);
        
        
    }
    return ($metric->getitem(0) / $metric->getitem(2), $metric->getitem(1) / $metric->getitem(2));


}

La siguiente función de entrenamiento luego entrena un modelo neten un conjunto de datos de entrenamiento al que se accede a través train_iterde varias épocas, que se especifica mediante num_epochs. Al final de cada época, el modelo se evalúa en un conjunto de datos de prueba al que se accede a través de test_iter. Aprovecharemos la Animatorclase para visualizar el progreso del entrenamiento.

In [25]:
# aqui va la funcion train_ch3

sub train_ch3{
    my ($net, $train_iter, $test_iter, $loss, $num_epochs, $updater) = @_;
    my $animator = d2l->animator(my $label='epoch',my $xlim=[1,$num_epochs],my $ylim=[0.3,0.9],my $leyend=['train loss','train acc','test acc']);
    my $train_metrics;
    my $test_acc;
    
    
    while(defined(my $epoch = mx->nd->arrange($num_epochs) )){
        
            $train_metrics = train_epoch_ch3($net, $train_iter, $loss, $updater);
            $test_acc = evaluate_accuracy($net, $train_iter, $loss, $updater);
            $animator->add($epoch + 1, $train_metrics + ($test_acc,))  
    }
    my ($train_loss, $train_acc) = $train_metrics;
    if($train_loss >= 0.5){
    
        die "Error: train_loss = ",$train_loss->asarray;
    }
     if(($train_acc > 1) && ($train_acc <= 0.7)){
    
        die "Error: train_acc =",$train_acc->asarray;
    }
     if(($test_acc > 1) && ($test_acc <= 0.7)){
    
        die "Error: test_acc = ",$test_acc->asarray;
    }
    


}


Warning: Subroutine train_ch3 redefined at reply input line 3.


Como implementación desde cero, usamos el descenso de gradiente estocástico de minilotes definido en la Sección 3.2 para optimizar la función de pérdida del modelo con una tasa de aprendizaje de 0.1.

In [17]:
our $lr = 0.1;
sub updater{

    my $batch_size = shift;
    our ($W, $b, $lr);

    my %params = (w=>$W, b=>$b);
    return d2l->sgd(\%params, $lr, $batch_size);
    
}

0.1

Ahora entrenamos el modelo con 10 épocas. Tenga en cuenta que tanto el número de épocas ( num_epochs) como la tasa de aprendizaje ( lr) son hiperparámetros ajustables. Al cambiar sus valores, podemos aumentar la precisión de clasificación del modelo.

In [26]:
our $num_epochs = 10;

my ($net, $animator) = train_ch3(\&net, $train_iter, $test_iter, \&cross_entropy, $num_epochs, \&updater, $batch_size);


Error: Can't locate object method "animator" via package "d2l" at reply input line 5.


In [24]:
$animator->plot;

Error: Can't call method "plot" on an undefined value at reply input line 1.


# 3.6.7. Predicción
Ahora que el entrenamiento está completo, nuestro modelo está listo para clasificar algunas imágenes. Dada una serie de imágenes, compararemos sus etiquetas reales (primera línea de salida de texto) y las predicciones del modelo (segunda línea de salida de texto).

In [ ]:
# aqui va el algoritmo para la prediccion
